## Import modules

In [1]:
import numpy as np
import open3d as o3d
from sklearn.ensemble import RandomForestClassifier
import pdal
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plyfile import PlyData, PlyElement
import gc
from itertools import chain
from sklearn.metrics import confusion_matrix, classification_report
import os
import joblib # Use to save model
from datetime import datetime, timezone
# from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from glob import glob
# from sklearn.model_selection import GridSearchCV
sns.set()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# ROOT_PLY = """/home/sspiegel/CapstoneData/Paris/Toronto_3D/L001.ply"""

## Load in features

In [3]:
ROOT = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/radial/testing/2025_11_11T18_12_Lille2_r_0_1_grid_0_02_features.npz"""
ROOT2 = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/radial/testing/2025_11_11T18_40_Lille2_r_0_2_grid_0_04_features.npz"""
ROOT3 = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/radial/testing/2025_11_11T19_04_Lille2_r_0_4_grid_0_08_features.npz"""
ROOT4 = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/radial/testing/2025_11_11T19_35_Lille2_r_0_8_grid_0_16_features.npz"""
ROOT5 = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/radial/testing/2025_11_11T19_58_Lille2_r_1_6_grid_0_32_features.npz"""
ROOT6 = """/home/sspiegel/CapstoneData/Paris/training_10_classes/pickleFiles/radial/testing/2025_11_11T20_20_Lille2_r_3_2_grid_0_64_features.npz"""

In [4]:
fileList = [ROOT,ROOT2, ROOT3,ROOT4,ROOT5, ROOT6]

## Get computed features

In [5]:
das = [np.load(r)["array2"] for r in fileList]
das = np.hstack(das)

xyz = np.load(fileList[0])["array1"]

cls = np.load(fileList[0])["array3"]

## Load in points and labels

## Combine road markings with Ground points

In [6]:
# cls[cls==2] = 1

In [7]:
# cls[cls > 1] -= 1

## Get columns

In [8]:
cols = ["EigenSum","omnivariance","entropy","linearity","planarity","sphericity","curvature","verticality1","verticality2","count"]

In [9]:
# cols1 = [f"""{a}_radius1""" for a in cols]
# cols2 = [f"""{a}_radius2""" for a in cols]
ff = []

for i in range(1, len(fileList) + 1):
    col = [f"""{a}_radius{i}""" for a in cols]
    ff += col
    

# allCols = cols1 + cols2
    
    
    

In [10]:
allCols = ['X', 'Y','Z'] + ff + ['label']

In [11]:
allAtrs = np.hstack((xyz,das, cls.reshape(-1, 1)))

In [12]:
del xyz, das, cls

## Create dataframe

In [13]:
total_dataframe = pd.DataFrame(allAtrs, columns=allCols)
total_dataframe["label"] = total_dataframe["label"].astype(int)
for i in range(1, len(fileList) + 1):
    total_dataframe[f"""count_radius{i}"""] = total_dataframe[f"""count_radius{i}"""].astype(int)
# total_dataframe["count_radius1"] = total_dataframe["count_radius1"].astype(int)
# total_dataframe["count_radius2"] = total_dataframe["count_radius2"].astype(int)

# total_dataframe["labelName"] = total_dataframe["label"].apply(labelPoints)

In [14]:
total_dataframe = total_dataframe.query("label != 0")
total_dataframe = total_dataframe.copy()

In [15]:
gc.collect()

0

In [16]:
# ss = ss[ss["count"] > 10]

In [17]:
total_dataframe["label"].max()

np.int64(9)

## Create PLY files with features (Only do if they don't already exist)

In [18]:
# for i in range(1, 7):
#     ls = [col for col in list(total_dataframe) if col.endswith(f"""radius{i}""")]
#     ls = ['X','Y','Z'] + ls + ['label']
#     partial_df = total_dataframe[ls]

#     tpsOut = []
#     for idx, tpe in partial_df.dtypes.to_dict().items():
#         if tpe == 'int64':
#             tpsOut.append((idx, 'i4'))
#         elif tpe == 'float64':
#             tpsOut.append((idx, 'f8'))
            
#     vertex_data = np.empty(allAtrs.shape[0], dtype=tpsOut)
    
#     for t in tpsOut:
#         vertex_data[t[0]] = partial_df[t[0]].values
    
        
#     el = PlyElement.describe(vertex_data, 'vertex')
    
#     # Create a PlyData object and write to a PLY file
#     # Set text=True for ASCII PLY, or text=False for binary PLY
#     PlyData([el], text=False).write(f"""/home/sspiegel/CapstoneData/Paris/Toronto_3D/PC_with_features/L002_features_radius{i}.ply""")

In [19]:
rf = joblib.load("""/home/sspiegel/CapstoneData/Paris/RF_models/2025_11_12_Trained_radial_RF_Lille.joblib""")

In [20]:
total_dataframe.reset_index(inplace = True)

In [21]:
total_dataframe.shape

(21222580, 65)

In [22]:
X = total_dataframe[ff].to_numpy()

In [23]:
y = total_dataframe["label"].to_numpy()

yPred = rf.predict(X)

/home/sspiegel/miniconda3/envs/masterEnv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   21.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:   55.7s finished


In [24]:
(yPred == y).sum() / y.shape[0]

np.float64(0.9148926756313323)

In [25]:
cmNorm = confusion_matrix(y, yPred, normalize='true')

In [26]:
indx = ["ground","building","signage","bollard","trash can","barrier","pedestrian","car","vegetation"]

In [27]:
cmDF = pd.DataFrame(cmNorm, columns=indx, index = indx)

In [35]:
cmDF.to_csv("./results/results_confustionMatrix.csv")

In [28]:
cmDF.round(2).to_markdown()

'|            |   ground |   building |   signage |   bollard |   trash can |   barrier |   pedestrian |   car |   vegetation |\n|:-----------|---------:|-----------:|----------:|----------:|------------:|----------:|-------------:|------:|-------------:|\n| ground     |     0.98 |       0    |      0    |      0    |        0    |      0    |         0    |  0.01 |         0.01 |\n| building   |     0.01 |       0.83 |      0    |      0    |        0    |      0.1  |         0    |  0    |         0.04 |\n| signage    |     0    |       0.25 |      0.67 |      0    |        0.01 |      0.02 |         0    |  0    |         0.06 |\n| bollard    |     0.17 |       0    |      0.69 |      0.09 |        0.01 |      0    |         0    |  0    |         0.03 |\n| trash can  |     0.05 |       0    |      0.04 |      0.01 |        0.5  |      0.13 |         0.02 |  0.13 |         0.12 |\n| barrier    |     0    |       0.08 |      0.12 |      0    |        0.02 |      0.44 |         0    |

In [36]:
rep = classification_report(y, yPred)

In [37]:
report_dict = classification_report(y, yPred, target_names=indx, output_dict=True)

df_report = pd.DataFrame(report_dict).transpose()

# 5. Print the DataFrame
df_report.round(2)

,precision,recall,f1-score,support
ground,0.99,0.98,0.98,12042099.00
building,0.99,0.83,0.91,7193259.00
signage,0.49,0.67,0.57,109906.00
bollard,0.13,0.09,0.11,7298.00
trash can,0.43,0.50,0.46,115885.00
barrier,0.03,0.44,0.05,54818.00
pedestrian,0.20,0.11,0.14,11233.00
car,0.80,0.92,0.86,770451.00
vegetation,0.61,0.82,0.70,917631.00
accuracy,0.91,0.91,0.91,0.91


In [38]:
df_report = df_report[["precision","recall","f1-score"]].round(2)


In [39]:
df_report.to_markdown()

'|              |   precision |   recall |   f1-score |\n|:-------------|------------:|---------:|-----------:|\n| ground       |        0.99 |     0.98 |       0.98 |\n| building     |        0.99 |     0.83 |       0.91 |\n| signage      |        0.49 |     0.67 |       0.57 |\n| bollard      |        0.13 |     0.09 |       0.11 |\n| trash can    |        0.43 |     0.5  |       0.46 |\n| barrier      |        0.03 |     0.44 |       0.05 |\n| pedestrian   |        0.2  |     0.11 |       0.14 |\n| car          |        0.8  |     0.92 |       0.86 |\n| vegetation   |        0.61 |     0.82 |       0.7  |\n| accuracy     |        0.91 |     0.91 |       0.91 |\n| macro avg    |        0.52 |     0.6  |       0.53 |\n| weighted avg |        0.96 |     0.91 |       0.93 |'

In [42]:
df_report.to_csv("./results/results_PrecisionReport.csv")